In [3]:
import os
dirpath = './Pfam_info'
print('Available files: ', os.listdir(dirpath))

Available files:  ['Aegerolysin.txt', 'Aerolysin.txt', 'Anemone_cytotox.txt', 'Bac_thur_toxin.txt', 'BB_PF.txt', 'Binary_toxB_2.txt', 'Clenterotox.txt', 'Colicin.txt', 'Endotoxin_C.txt', 'ETX_MTX2.txt', 'FB_lectin.txt', 'Gasdermin.txt', 'Hemolysin_N.txt', 'HlyE.txt', 'Leukocidin.txt', 'MACPF.txt', 'PlyB_C.txt', 'PorB.txt', 'T4BSS_DotI_IcmL.txt', 'Tcda_Tcdb_pore.txt', 'Thiol_cytolysin.txt', 'Toxin_Tolip.txt']


In [4]:
in_files = []
for file in os.listdir(dirpath):
    f = './Pfam_info/' + file
    in_files.append(f)

In [5]:
in_files

['./Pfam_info/Aegerolysin.txt',
 './Pfam_info/Aerolysin.txt',
 './Pfam_info/Anemone_cytotox.txt',
 './Pfam_info/Bac_thur_toxin.txt',
 './Pfam_info/BB_PF.txt',
 './Pfam_info/Binary_toxB_2.txt',
 './Pfam_info/Clenterotox.txt',
 './Pfam_info/Colicin.txt',
 './Pfam_info/Endotoxin_C.txt',
 './Pfam_info/ETX_MTX2.txt',
 './Pfam_info/FB_lectin.txt',
 './Pfam_info/Gasdermin.txt',
 './Pfam_info/Hemolysin_N.txt',
 './Pfam_info/HlyE.txt',
 './Pfam_info/Leukocidin.txt',
 './Pfam_info/MACPF.txt',
 './Pfam_info/PlyB_C.txt',
 './Pfam_info/PorB.txt',
 './Pfam_info/T4BSS_DotI_IcmL.txt',
 './Pfam_info/Tcda_Tcdb_pore.txt',
 './Pfam_info/Thiol_cytolysin.txt',
 './Pfam_info/Toxin_Tolip.txt']

In [6]:
len(in_files)

22

In [13]:
from tqdm import tqdm

num_structures = 0
labels = []

for filename in tqdm(in_files):
    
    #PART I
    
    pdbid = []
    chain = []
    pdbres = []
    with open(filename, "r") as f:
        lines = f.readlines()
        total = len(lines)
        dummy_pdbid = []
        dummy_chain = []
        dummy_pdbres = []
        count = 1
        for line in lines:
            elements = line.split('\t')
            if len(elements) == 3: #when we have only three columns, the only difference is the chain ID, the uniprot ID 
                #and the PDB id are the same, so we do not create a new array (inside the big array), but we just append
                dummy_pdbid.append(dummy_pdbid[-1])
                dummy_chain.append(elements[0])
                start = elements[1].split('-')[0]
                end = elements[1].split('-')[1]
                dummy_pdbres.append((int(start.strip()),int(end.strip())))
            elif len(elements) == 4: #NOW EACH NEW PDB ID (ENTRY) WILL COUNT EVEN IF FOR SAME UNIPROT SEQ
                #CAN BE EASILY CHANGED BY REVERTING TO STRUCTURE ABOVE (SEE LEN(ELEMENTS)==3 STRUCTURE)
                if len(dummy_pdbid) != 0:
                    pdbid.append(dummy_pdbid)
                    chain.append(dummy_chain)
                    pdbres.append(dummy_pdbres)
                    dummy_pdbid = []
                    dummy_chain = []
                    dummy_pdbres = []
                dummy_pdbid.append(elements[0])
                dummy_chain.append(elements[1])
                start = elements[2].split('-')[0]
                end = elements[2].split('-')[1]
                dummy_pdbres.append((int(start.strip()),int(end.strip())))
            elif len(elements) == 5: #NOW EACH NEW PDB ENTRY WILL COUNT EVEN IF FOR SAME UNIPROT SEQ
                #LEN 5 HAPPENS WHEN MULTIPLE DOMAIN OF THE SAME UNIPROT ENTRY ARE IN FAMILY
                #https://pfam.xfam.org/family/aerolysin#tabview=tab9
                if len(dummy_pdbid) != 0:
                    pdbid.append(dummy_pdbid)
                    chain.append(dummy_chain)
                    pdbres.append(dummy_pdbres)
                    dummy_pdbid = []
                    dummy_chain = []
                    dummy_pdbres = []
                dummy_pdbid.append(elements[1])
                dummy_chain.append(elements[2])
                start = elements[3].split('-')[0]
                end = elements[3].split('-')[1]
                dummy_pdbres.append((int(start.strip()),int(end.strip())))
            elif len(elements) == 6:
                if len(dummy_pdbid) != 0:
                    pdbid.append(dummy_pdbid)
                    chain.append(dummy_chain)
                    pdbres.append(dummy_pdbres)
                    dummy_pdbid = []
                    dummy_chain = []
                    dummy_pdbres = []
                dummy_pdbid.append(elements[2])
                dummy_chain.append(elements[3])
                start = elements[4].split('-')[0]
                end = elements[4].split('-')[1]
                dummy_pdbres.append((int(start.strip()),int(end.strip())))
            else:
                print(len(elements))
                print(line)
            if (count == total):
                pdbid.append(dummy_pdbid)
                chain.append(dummy_chain)
                pdbres.append(dummy_pdbres)
            count = count + 1
   
    #PART II
    #totally could do binary search since ss.txt is in order

    array = []
    for i, j in zip(pdbid, chain):
        dummy = []
        for k,l in zip(i, j):
            dummy.append(">"+k+":"+l+":sequence")
        array.append(dummy)

    mapfile = open("ss.txt", "r")
    maplines = mapfile.readlines()
    
    structures = []
    #idxes_1 = []
    #idxes_2 = []
    for uniprot in array: #what this is doing is chosing an array from a nested array and the chosen array corresponds 
        #to a new PDB ID with many different chain IDs and the one added to labels is the one found first in ss.txt
        idx_1 = array.index(uniprot)
        structure = []
        count = 0
        total_length = len(maplines)
        for line in maplines:
            if line.strip() in uniprot:
                #print(line.strip())
                idx_2 = uniprot.index(line.strip())
                #idxes_1.append(idx_1)
                #idxes_2.append(idx_2)
                labels.append(array[idx_1][idx_2][1:7])
                while not (maplines[count+1].startswith(">")):
                    structure.append(maplines[count+1].strip())
                    count = count + 1
                structures.append(''.join(structure))
                break;
            #if count == total_length-1:
                #print("In file: ", filename, " Entry: ", uniprot, " was not found in mapping file!")
            count = count + 1
    mapfile.close()
    length = len(structures)
    
    #PFAM PDB DOMAIN ANNOTATION IS WRONG
    #https://pfam.xfam.org/family/PF03944#tabview=tab9
    #LAST ENTRY 4D8M HAS PDB RESIDUE 697 IN DOMAIN, BUT
    #4D8M IS ONLY 587 LONG! 
    #https://www.rcsb.org/structure/4d8m
    
    #not using idxes, but checking all different pdb structures 
    #(we do not differentiate between chains)
    #even if it cannot be found in ss.txt
    #see how to use idxes in test.ipynb
    
    res = []
    pdbchain = []
    pdbentry = []
    for i in range(len(pdbid)):
        chosen_res = pdbres[i][0]
        chosen_entry = pdbid[i][0]
        chosen_chain = chain[i][0]
        res.append(chosen_res)
        pdbchain.append(chosen_chain)
        pdbentry.append(chosen_entry)
        
    info = []
    for q,w,e in zip(pdbentry, pdbchain, res):
        text = q + ' ' + w + ' ' + ':'.join([str(num) for num in e])
        info.append(text)
    
    file_out = './structures/posAAsequences_' + filename.split('/')[-1]
    with open(file_out, 'w+') as result_file:
        result_file.write('\n'.join(structures))
        
    num_structures += length
    

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [04:10<00:00, 11.38s/it]


In [8]:
num_structures

372

In [9]:
array[0][0][1:7]

'1NTN:A'

In [10]:
len(labels)

372

In [11]:
import pickle
path = "./" + "pos_pdb_ids" + ".pickle"
output = open(path, 'w+b')
pickle.dump(labels, output)
output.close()